# Holiday Manager

## Pseudocode
We are looking to create an application which manages holidays. Before we begin with the application, we will pre-load the holidays from https://www.timeanddate.com/holidays/us/2022?hol=33554809 into a JSON file along with the pre-loaded holidays already loaded in the JSON file.

With the holidays approaching, the marketing person may want to know what the weather looks like so they can determine what to market more. Use a weather API (e.g. [Open Weather Map API](https://rapidapi.com/community/api/open-weather-map)) to show what the weather looks like for the current period (more in the UI explanation below).

**Start Up**
* Display welcome message/UI info
* Display number of holidays stored in the system (e.g. JSON file)

**Main Menu**
* Display message Holiday menu
* Allow user to choose from:
> 1. Add a Holiday
> 2. Remove a Holiday
> 3. Save Holiday List
> 4. View Holidays
> 5. Exit

**Add a Holiday**
* Display Add a Holiday
* Prompt the user for Holiday name and Date
> * If date is typed incorrectly (i.e. doesn't fit into ISO 8601 YYYY-MM-DD), display error message 'Invalid date' and reprompt
> * If date is typed correctly, display success message '... has been added to list' and add holiday to list

**Remove a Holiday**
* Display Remove a Holiday
* Prompt user for Holiday Name
> * If user inputs a holiday not in file, display error message 'HolidayName not found' and reprompt
> * If user inputs holiday in file, display success message and remove from holiday list

**Save Holiday List**
* Display Save Holiday List
* Prompt user if they want to save their changes
> * If no, do not save and display canceled message 'Holiday list file save cancelled'
> * If yes, save holiday list to JSON and display success message 'Changes saved'

**View Holidays**
* Display View Holidays
* Prompt user for year
* Prompt user for week#, leave blank if current week
> * if week# blank, prompt user if they would like to see this week's weather
> * Display holidays for selected week and weather (if applies) using lambda expression

**Exit**
* Display exit 
* Prompt user if they are sure they want to exit
> * if yes, display 'Goodbye' and exit

### Holidays
We'll gather all the holidays and store in JSON.
* Make sure to represent holidays as objects in Python
* The List of Holidays should be represented as an object in Python
* The holiday class should include a way to display a holiday in the following format: `HolidayName (Date)`

In [1]:
from datetime import datetime
from datetime import timedelta
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass
import inspect
from config import jsonlocation
from config import weatherurl
from config import 


In [2]:
def getHTML(url):
    response = requests.get(url)
    return response.text

In [3]:
##
@dataclass
class Holiday:
    name: str
    date: datetime
        
    def __str__ (self):
        # String output
        date_format = '%Y-%m-%d'
        date_str = datetime.strftime(self.date, date_format)
        # Holiday output when printed.
        return '%s (%s) ' % (self.name, date_str)

In [4]:
# -------------------------------------------
# The HolidayList class acts as a wrapper and container
# For the list of holidays
# Each method has pseudo-code instructions
# --------------------------------------------
class HolidayList:
    def __init__(self):
        self.innerHolidays = []
   
    def addHoliday(self, holidayObj):
        #Make sure holidayObj is not already in list
        found_hol = self.findHoliday(holidayObj.name, holidayObj.date)
        if found_hol != None:
            pass
            #print(f'Error: {holidayObj} already exists in list')
            
        elif found_hol == None:            
            # Make sure holidayObj is a Holiday Object by checking the type
            if isinstance(holidayObj, Holiday):
                # Use innerHolidays.append(holidayObj) to add holiday
                self.innerHolidays.append(holidayObj)
            else:
                print('Error: Please try again.')

    def findHoliday(self, HolidayName, Date):
        # Find Holiday in innerHolidays
        found_holiday = [x for x in self.innerHolidays if x.name == HolidayName and x.date == Date]
        # Return Holiday object
        try:
            found_hol_obj = found_holiday[0]
        except IndexError:
            found_hol_obj = None
        return found_hol_obj

    def removeHoliday(self, holidayObj):
        # Holiday has already been found in innerList using the RemoveHoliday() function
        # remove the Holiday from innerHolidays
        self.innerHolidays = list(filter(lambda x : x != holidayObj, self.innerHolidays))
        
    def read_json(self, jsonlocation):
        # Read in things from json file location
        with open(jsonlocation, 'r') as f:
            data = json.load(f)
            f.close()
        # Use addHoliday function to add holidays to inner list.
        for holiday in data['holidays']:
            holiday_name = holiday['name']
            holidate = holiday['date']
            date_format = '%Y-%m-%d'
            formatted_date = datetime.strptime(holidate, date_format).date()
            holidayobj = Holiday(holiday_name, formatted_date)
            self.addHoliday(holidayobj)

    def save_to_json(self, jsonlocation):
        #Makee empty holiday dictionary list
        holiday_dict_list= {"holidays" : []}
        #Gather individual holiday objects from innerHoliday list and append to holiday dict list
        for holiday in self.innerHolidays:
            holiday_name = holiday.name
            holidate = holiday.date
            date_format = '%Y-%m-%d'
            date = datetime.strftime(holidate, date_format)
            holiday_dict = {'name': holiday_name, 'date': date}
            holiday_dict_list['holidays'].append(holiday_dict)
        # Write out json file to selected file.
        with open(jsonlocation, 'w') as f:
            json.dump(holiday_dict_list, f, sort_keys = True, indent = 4)
            f.close()
        
    def scrapeHolidays(self):
        current_year = int(datetime.now().isocalendar()[0])
        #Years from current year-2 to current year+2
        for year in range(current_year-2, current_year+3):
            year_str = str(year)
            # Scrape Holidays from https://www.timeanddate.com/holidays/us/2022?hol=33554809
            html = getHTML(f"https://www.timeanddate.com/holidays/us/{year_str}?hol=33554809")
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find('table', attrs = {'id' : 'holidays-table'})
            #Get each holiday name and date
            for table_entry in table.find_all('tr', attrs = {'class' : 'showrow'}):
                holiday = table_entry.find('a').string
                date = table_entry.find('th').string
                holiday_date = year_str + ' ' + date
                date_format = '%Y %b %d'
                holidate = datetime.strptime(holiday_date, date_format).date()
                # Check to see if name and date of holiday is in innerHolidays array done in addHoliday method
                hol_obj = Holiday(holiday, holidate)
                self.addHoliday(hol_obj)   

    def numHolidays(self):
        # Return the total number of holidays in innerHolidays
        return len(self.innerHolidays)
    
    def filter_holidays_by_week(self, year, week_number):
        # Use a Lambda function to filter innerHolidays by week number and year
        filtered_hol_list = list(
            filter(
                lambda x : x.date.isocalendar()[1] == week_number and x.date.isocalendar()[0] == year, self.innerHolidays
            )
        )
        # return your filtered holidays
        return filtered_hol_list

    def displayHolidaysInWeek(self, year, week_number):
        # Use your filter_holidays_by_week to get list of holidays within a week as a parameter
        filt_hol_list = self.filter_holidays_by_week(year, week_number)
        # Output formated holidays in the week.
        for hol in filt_hol_list:
            print(hol)

    def getWeather(weekNum):
        pass
        # Convert weekNum to range between two days
        # Use Try / Except to catch problems
        # Query API for weather in that week range
        # Format weather information and return weather string.

    def viewCurrentWeek(self):
        # Use the Datetime Module to look up current week and year
        current_year = datetime.now().isocalendar()[0]
        current_week = datetime.now().isocalendar()[1]
        # Use your filter_holidays_by_week function to get the list of holidays
        #current_week_hol = self.innerHolidays.filter_holidays_by_week(current_year, current_week)
        # Use your displayHolidaysInWeek function to display the holidays in the week
        self.displayHolidaysInWeek(current_year, current_week)
        # Ask user if they want to get the weather
        #get_weather = str(input('Would you like to see the weather? [y/n]: ')).lower()
        # If yes, use your getWeather function and display results
        #if get_weather == 'y':
        #    for hol in 

In [5]:
#UI Start Up
##
def StartUp(holiday_list):
    #Find number of holidays stored in system
    num_holidays = holiday_list.numHolidays()
    print(f'''
        Holiday Management
        =================================
        There are {num_holidays} holidays stored in the system
    ''')
    
    MainMenu(holiday_list)

In [6]:
#UI MainMenu
##
def MainMenu(holiday_list):
    print(f'''
        Holiday Menu
        =====================================
    ''')
    #Store user selection from Main Menu
    getting_input = True
    while getting_input:
        try:
            user_selection = int(input('''
                1. Add a Holiday
                2. Remove a Holiday
                3. Save Holiday List
                4. View Holidays
                5. Exit
            '''))
            getting_input = False
        except:
            print('Enter the number again')
    #Point to appropriate directory
    if user_selection == 1:
        AddHoliday(holiday_list)
    elif user_selection == 2:
        RemoveHoliday(holiday_list)
    elif user_selection == 3:
        SaveHolidayList(holiday_list)
    elif user_selection == 4:
        ViewHolidays(holiday_list)
    elif user_selection == 5:
        Exit(holiday_list)


In [7]:
#UI Add Holiday
##
def AddHoliday(holiday_list):
    #Print welcome message
    print(f'''
        Add a Holiday
        ==============
    ''')
    #Gather Holiday name from user
    holiday_name = str(input("Holiday: "))
    #Gather Holiday date from user and ensure datetime
    incorrect_form = True
    while incorrect_form:
        try:
            testdate = input('Enter date (YYYY-MM-DD): ')
            date_format = '%Y-%m-%d'
            date = datetime.strptime(testdate, date_format).date()
        except ValueError:
            print('Invalid date. Please try again.')
        else:
            incorrect_form = False
            #Check if Holiday already exists in list
            found_holiday = holiday_list.findHoliday(holiday_name, date)
            #If exists, let user know
            if isinstance(found_holiday, Holiday):
                print(f'''
                    Error:
                    {holiday_name} already exists.
                ''')
            #If not found, addHoliday
            elif found_holiday == None:
                holiday_object = Holiday(holiday_name, date)
                holiday_list.addHoliday(holiday_object)
                # print to the user that you added a holiday
                print(f'Success: {holiday_object} has been added to the holiday list.')
                global changes_saved
                changes_saved = False
            
    #MainMenu()

In [8]:
#UI Remove Holiday
##
def RemoveHoliday(holiday_list):
    #Print welcome message
    print(f'''
        Remove a Holiday
        =================
    ''')
    holiday_not_in_list = True
    while holiday_not_in_list:
        #Gather holiday name from user
        holiday_name = str(input('Holiday Name: '))
        #Gather Holiday date from user and ensure datetime
        incorrect_form = True
        while incorrect_form:
            try:
                testdate = input('Enter date (YYYY-MM-DD): ')
                date_format = '%Y-%m-%d'
                date = datetime.strptime(testdate, date_format).date()
            except ValueError:
                print('Invalid date. Please try again.')
            else:
                incorrect_form = False
    
        #Check if holiday_name is in list
        found_holiday_obj = holiday_list.findHoliday(holiday_name, date)
        if isinstance(found_holiday_obj, Holiday):
            holiday_not_in_list = False
            holiday_list.removeHoliday(found_holiday_obj)
            # inform user you deleted the holiday
            print(f'''
                Success:
                {found_holiday_obj} has been removed from the list.
            ''')

            global changes_saved
            changes_saved = False
        elif found_holiday_obj == None:
            print(f'''
                Error:
                {holiday_name} not found.
            ''')
    
    
    #MainMenu()

In [9]:
#UI Save Holiday List
##
def SaveHolidayList(holiday_list):
    #Print welcome message
    print(f'''
        Save Holiday List
        ===================
    ''')
    #User save input
    user_save = str(input('Are you sure you want to save your changes? [y/n]: ')).lower()
    #Check user response
    if user_save == 'n':
        print('''
            Cancelled:
            Holiday list file save cancelled.
        ''')
    elif user_save == 'y':
        #Save list
        holiday_list.save_to_json(jsonlocation)
        global changes_saved
        changes_saved = True
        print('''
            Success:
            Your Changes have been saved
        ''')
    
    
    #MainMenu()

In [10]:
#UI View Holidays

def ViewHolidays(holiday_list):
    #Print welcome message
    print(f'''
        View Holidays
        ==============
    ''')
    #User save input
    holiday_year = int(input('Which year?: '))
    #Ensure year is within [current year-2, current_year+2]
    holiday_week = input('Which week? #[1-52, Leave blank for the current week]: ')
    #Print holidays in week
    if holiday_week == '':
        print(f'These are the holidays for this week:')
        holiday_list.viewCurrentWeek()
        
    else:
        holiday_week = int(holiday_week)
        print(f'These are the holidays for {holiday_year} week#{holiday_week}:')
        holiday_list.displayHolidaysInWeek(holiday_year, holiday_week)
    
    
    #MainMenu()

In [11]:
#UI Exit
##
def Exit(holiday_list):
    #Print welcome message
    print(f'''
        Exit
        =====
    ''')
    #Check if changes need to be saved
    global changes_saved
    #User exit input
    if changes_saved == True:
        user_exit = str(input('Are you sure you want to exit? [y/n]: ')).lower()
    elif changes_saved == False:
        user_exit = str(input('''
            Are you sure you want to exit?
            Your changes will be lost.
            [y/n]: 
            ''')).lower()
    
    if user_exit == 'n':
        MainMenu(holiday_list)
    elif user_exit == 'y':
        print('Goodbye!')
        global user_using
        user_using = False
        

In [12]:
#Main set-up
def main():
    # Large Pseudo Code steps
    # -------------------------------------
    # 1. Initialize HolidayList Object
    holiday_list = HolidayList()
    # 2. Load JSON file via HolidayList read_json function
    holiday_list.read_json(jsonlocation)
    # 3. Scrape additional holidays using your HolidayList scrapeHolidays function.
    holiday_list.scrapeHolidays()
    # 3. Create while loop for user to keep adding or working with the Calender
    global user_using
    user_using = True
    while user_using == True:
        # 4. Display User Menu (Print the menu)
        StartUp(holiday_list)
        

    # 5. Take user input for their action based on Menu and check the user input for errors
    # 6. Run appropriate method from the HolidayList object depending on what the user input is
    # 7. Ask the User if they would like to Continue, if not, end the while loop, ending the program.  If they do wish to continue, keep the program going. 



In [13]:
#Start app
if __name__ == "__main__":
    main();


        Holiday Management
        There are 627 holidays stored in the system
    

        Holiday Menu
    

                1. Add a Holiday
                2. Remove a Holiday
                3. Save Holiday List
                4. View Holidays
                5. Exit
            1

        Add a Holiday
    
Holiday: Christmas Day
Enter date (YYYY-MM-DD): 2022-12-25

                    Error:
                    Christmas Day already exists.
                

        Holiday Management
        There are 627 holidays stored in the system
    

        Holiday Menu
    

                1. Add a Holiday
                2. Remove a Holiday
                3. Save Holiday List
                4. View Holidays
                5. Exit
            1

        Add a Holiday
    
Holiday: Mike
Enter date (YYYY-MM-DD): 2023-02-07
Success: Mike (2023-02-07)  has been added to the holiday list.

        Holiday Management
        There are 628 holidays stored in the system
    

        Ho

In [13]:
#Initiate holiday_list instance
a_holiday_list = HolidayList()

In [14]:
a_holiday_list.read_json(jsonlocation)

Margaret Thatcher Day (2021-01-10)  entered
World Sketchnote Day (2021-01-11)  entered
Zanzibar Revolution Day (2021-01-12)  entered
National Rubber Ducky Day (2021-01-13)  entered
Tamil Thai Pongdal Day (2021-01-14)  entered
National Bagel Day (2021-01-15)  entered
Signing of the Peace Accords (2021-01-16)  entered
Christmas Day (2022-12-25)  entered


In [15]:
a_holiday_list.scrapeHolidays()

New Year's Day (2020-01-01)  entered
Stephen Foster Memorial Day (2020-01-13)  entered
Martin Luther King Jr. Day (2020-01-20)  entered
Lunar New Year (2020-01-25)  entered
Kansas Day (2020-01-29)  entered
National Freedom Day (2020-02-01)  entered
First Day of Black History Month (2020-02-01)  entered
Groundhog Day (2020-02-02)  entered
Super Bowl (2020-02-02)  entered
National Girls and Women in Sports Day (2020-02-05)  entered
National Wear Red Day (2020-02-07)  entered
Valentine's Day (2020-02-14)  entered
Presidents' Day (2020-02-17)  entered
Shrove Tuesday/Mardi Gras (2020-02-25)  entered
St. David's Day (2020-03-01)  entered
First Day of Women's History Month (2020-03-01)  entered
First Day of Irish American Heritage Month (2020-03-01)  entered
Read Across America Day (2020-03-02)  entered
Super Tuesday (2020-03-03)  entered
Employee Appreciation Day (2020-03-06)  entered
St. Patrick's Day (2020-03-17)  entered
March Equinox (2020-03-19)  entered
National Vietnam War Veterans Da

New Year's Day (2022-01-01)  entered
Stephen Foster Memorial Day (2022-01-13)  entered
Martin Luther King Jr. Day (2022-01-17)  entered
Kansas Day (2022-01-29)  entered
National Freedom Day (2022-02-01)  entered
Lunar New Year (2022-02-01)  entered
First Day of Black History Month (2022-02-01)  entered
Groundhog Day (2022-02-02)  entered
National Girls and Women in Sports Day (2022-02-02)  entered
National Wear Red Day (2022-02-04)  entered
Super Bowl (2022-02-13)  entered
Valentine's Day (2022-02-14)  entered
Presidents' Day (2022-02-21)  entered
Shrove Tuesday/Mardi Gras (2022-03-01)  entered
St. David's Day (2022-03-01)  entered
First Day of Women's History Month (2022-03-01)  entered
First Day of Irish American Heritage Month (2022-03-01)  entered
Read Across America Day (2022-03-02)  entered
Employee Appreciation Day (2022-03-04)  entered
St. Patrick's Day (2022-03-17)  entered
March Equinox (2022-03-20)  entered
National Vietnam War Veterans Day (2022-03-29)  entered
Doctors' Day

New Year's Day (2024-01-01)  entered
Stephen Foster Memorial Day (2024-01-13)  entered
Martin Luther King Jr. Day (2024-01-15)  entered
Kansas Day (2024-01-29)  entered
National Freedom Day (2024-02-01)  entered
First Day of Black History Month (2024-02-01)  entered
Groundhog Day (2024-02-02)  entered
National Wear Red Day (2024-02-02)  entered
National Girls and Women in Sports Day (2024-02-07)  entered
Lunar New Year (2024-02-10)  entered
Super Bowl (2024-02-11)  entered
Shrove Tuesday/Mardi Gras (2024-02-13)  entered
Valentine's Day (2024-02-14)  entered
Presidents' Day (2024-02-19)  entered
St. David's Day (2024-03-01)  entered
Read Across America Day (2024-03-01)  entered
Employee Appreciation Day (2024-03-01)  entered
First Day of Women's History Month (2024-03-01)  entered
First Day of Irish American Heritage Month (2024-03-01)  entered
St. Patrick's Day (2024-03-17)  entered
March Equinox (2024-03-19)  entered
National Vietnam War Veterans Day (2024-03-29)  entered
Doctors' Day

In [15]:
#Add holiday to holiday_list
AddHoliday(a_holiday_list)


        Add a Holiday
    
Holiday: Mikes
Enter date (YYYY-MM-DD): 2023-02-07
Mikes (2023-02-07)  entered
Success: Mikes (2023-02-07)  has been added to the holiday list.


In [16]:
a_holiday_list.innerHolidays

[Holiday(name='Margaret Thatcher Day', date=datetime.date(2021, 1, 10)),
 Holiday(name='World Sketchnote Day', date=datetime.date(2021, 1, 11)),
 Holiday(name='Zanzibar Revolution Day', date=datetime.date(2021, 1, 12)),
 Holiday(name='National Rubber Ducky Day', date=datetime.date(2021, 1, 13)),
 Holiday(name='Tamil Thai Pongdal Day', date=datetime.date(2021, 1, 14)),
 Holiday(name='National Bagel Day', date=datetime.date(2021, 1, 15)),
 Holiday(name='Signing of the Peace Accords', date=datetime.date(2021, 1, 16)),
 Holiday(name='Christmas Day', date=datetime.date(2022, 12, 25)),
 Holiday(name='Mikes', date=datetime.date(2023, 2, 7))]

In [17]:
RemoveHoliday(a_holiday_list)


        Remove a Holiday
    
Holiday Name: Mikes
Enter date (YYYY-MM-DD): 2023-02-07

                Success:
                Mikes (2023-02-07)  has been removed from the list.
            


In [18]:
current_year = int(datetime.now().isocalendar()[0])
for year in range(current_year-2, current_year+3):
    year_str = str(year)
    print(year_str)

2020
2021
2022
2023
2024


In [32]:
ViewHolidays(a_holiday_list)


        View Holidays
    
Which year?: 2021
Which week? #[1-52, Leave blank for the current week]: 2
These are the holidays for 2021 week#2:
World Sketchnote Day (2021-01-11) 
Zanzibar Revolution Day (2021-01-12) 
National Rubber Ducky Day (2021-01-13) 
Tamil Thai Pongdal Day (2021-01-14) 
National Bagel Day (2021-01-15) 
Signing of the Peace Accords (2021-01-16) 


In [12]:
a_holiday_list.filter_holidays_by_week(2021, 2)

[Holiday(name='World Sketchnote Day', date=datetime.date(2021, 1, 11)),
 Holiday(name='Zanzibar Revolution Day', date=datetime.date(2021, 1, 12)),
 Holiday(name='National Rubber Ducky Day', date=datetime.date(2021, 1, 13)),
 Holiday(name='Tamil Thai Pongdal Day', date=datetime.date(2021, 1, 14)),
 Holiday(name='National Bagel Day', date=datetime.date(2021, 1, 15)),
 Holiday(name='Signing of the Peace Accords', date=datetime.date(2021, 1, 16))]

In [11]:
#Create a dict from HolidayList obj

holiday_dict_list= {"holidays" : []}
#print(type(holiday_dict_list['holidays']))
#print(a_holiday_list.innerHolidays[0])

for holiday in a_holiday_list.innerHolidays:
    holiday_name = holiday.name
    holidate = holiday.date
    date_format = '%Y-%m-%d'
    date = datetime.strftime(holidate, date_format)
    holiday_dict = {'name': holiday_name, 'date': date}
    holiday_dict_list['holidays'].append(holiday_dict)
    #holidayobj = Holiday(holiday_name, date)
    #a_holiday_list.addHoliday(holidayobj)
    

#json_test = json.dumps(a_holiday_list.innerHolidays)
#print(json_test)

In [58]:
#Remove holiday from holiday list
RemoveHoliday(a_holiday_list)


        Remove a Holiday
    
Holiday Name: St Patties Day
Enter date (YYYY-MM-DD): 2022-03-17

                Error:
                St Patties Day not found.
            
Holiday Name: New Year's Day
Enter date (YYYY-MM-DD): 2022-01-01

            Success:
            New Year's Day has been removed from the list.
        


In [10]:
#    def getWeather(weekNum):
#        pass
        # Convert weekNum to range between two days
        # Use Try / Except to catch problems
        # Query API for weather in that week range
        # Format weather information and return weather string.

In [17]:
def getDateRangeFromWeek(year, week_num):
    #Get first day
    first_week_day = datetime.strptime(f'{year}-W{week_num}-1', '%Y-W%W-%w')
    first_day = first_week_day.date()
    #Get second day
    last_week_day = first_day + timedelta(days=6.9)
    #return the range
    return first_day, last_week_day

In [18]:
current_year = datetime.now().isocalendar()[0]
current_week = datetime.now().isocalendar()[1]
first_day, last_day = getDateRangeFromWeek(current_year, current_week)
print(first_day, last_day)

2022-07-11 2022-07-17


In [16]:
datetime.now().isocalendar()

datetime.IsoCalendarDate(year=2022, week=28, weekday=1)

In [14]:
import requests

url = "https://community-open-weather-map.p.rapidapi.com/forecast/daily"

querystring = {"q":"san francisco,us","lat":"35","lon":"139","cnt":"10","units":"imperial"}

headers = {
	"X-RapidAPI-Key": "064346a1dcmshb79799426712a68p1a48f2jsn42c6f7f6a9d7",
	"X-RapidAPI-Host": "community-open-weather-map.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"city":{"id":5391959,"name":"San Francisco","coord":{"lon":-122.4194,"lat":37.7749},"country":"US","population":805235,"timezone":-25200},"cod":"200","message":0.0366846,"cnt":10,"list":[{"dt":1657569600,"sunrise":1657544217,"sunset":1657596781,"temp":{"day":67.17,"min":56.43,"max":67.17,"night":59.45,"eve":61.38,"morn":58.01},"feels_like":{"day":66.79,"night":59.27,"eve":61.25,"morn":57.56},"pressure":1013,"humidity":68,"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03d"}],"speed":13.58,"deg":251,"gust":16.84,"clouds":50,"pop":0},{"dt":1657656000,"sunrise":1657630657,"sunset":1657683157,"temp":{"day":65.61,"min":56.46,"max":65.62,"night":58.53,"eve":61.75,"morn":57.4},"feels_like":{"day":65.01,"night":58.26,"eve":61.3,"morn":56.97},"pressure":1013,"humidity":67,"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03d"}],"speed":13.2,"deg":256,"gust":17.78,"clouds":35,"pop":0},{"dt":1657742400,"sunrise":1657717097,"sunset":165776

In [15]:
weather_list = response.json()

In [28]:
weather_list['list'][0]

{'dt': 1657569600,
 'sunrise': 1657544217,
 'sunset': 1657596781,
 'temp': {'day': 67.17,
  'min': 56.43,
  'max': 67.17,
  'night': 59.45,
  'eve': 61.38,
  'morn': 58.01},
 'feels_like': {'day': 66.79, 'night': 59.27, 'eve': 61.25, 'morn': 57.56},
 'pressure': 1013,
 'humidity': 68,
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'speed': 13.58,
 'deg': 251,
 'gust': 16.84,
 'clouds': 50,
 'pop': 0}

In [32]:
for day in weather_list['list'][0:7]:
    day_weather = day['weather'][0]['description']
    print(day_weather)

scattered clouds
scattered clouds
few clouds
few clouds
few clouds
few clouds
few clouds
